In [112]:
from qiskit_ibm_runtime import QiskitRuntimeService
QiskitRuntimeService.save_account(channel="ibm_cloud",
                                  overwrite=True, 
                                  token="RXxwIjlu0I1zY4I8ia1M1dMek5nPJ85gT0bpPfSIffkl",
                                  instance="crn:v1:bluemix:public:quantum-computing:us-east:a/628fd5eb6d2144f1a12579686531c6b7:5fa1452a-e2f4-46b1-8c00-184731a785b9::")

In [113]:
# Cell 1: Imports and Setup
import os, time, json, numpy as np, sys
import warnings
warnings.filterwarnings('ignore')

# Core Qiskit imports
from qiskit import QuantumCircuit, transpile
from qiskit.quantum_info import SparsePauliOp
from qiskit.circuit.library import QAOAAnsatz
from qiskit_aer import AerSimulator
import matplotlib.pyplot as plt
from qiskit_algorithms.optimizers import SPSA, COBYLA

IBM_PROVIDER_AVAILABLE = False
IBM_RUNTIME_AVAILABLE = False
ALGORITHMS_AVAILABLE = False
# Try different import approaches for IBM services
try:
    from qiskit_ibm_runtime import QiskitRuntimeService, Session
    from qiskit.primitives import Estimator, Sampler
    print("✓ qiskit_ibm_runtime imported successfully")
    IBM_RUNTIME_AVAILABLE = True
except ImportError:
    print("⚠ qiskit_ibm_runtime not available")
    IBM_RUNTIME_AVAILABLE = False

    try:
        from qiskit_ibm_provider import IBMProvider
        print("✓ qiskit_ibm_provider imported successfully")
        IBM_PROVIDER_AVAILABLE = True
    except ImportError:
        print("⚠ qiskit_ibm_provider not available")
        IBM_PROVIDER_AVAILABLE = False

# Try optimizers
try:
    from qiskit_algorithms.optimizers import SPSA, COBYLA
    from qiskit_algorithms.minimum_eigensolvers import QAOA
    print("✓ qiskit_algorithms imported successfully")
    ALGORITHMS_AVAILABLE = True
    print("ALGORITHMS_AVAILABLE:", ALGORITHMS_AVAILABLE)
except ImportError:
    print("⚠ qiskit_algorithms not available, will use alternative implementations")
    ALGORITHMS_AVAILABLE = False

# Add your backend path
sys.path.append(r"D:\AQVH\backend")

# Import your custom modules
try:
    from vrp_solver import VRPQUBOFormulation
    from classical_solver import solve_vrp_classical
    from sample_data import get_test_case, validate_solution
    print("✓ Custom VRP modules imported successfully")
    CUSTOM_MODULES_AVAILABLE = True
except ImportError as e:
    print(f"⚠ Custom modules not available: {e}")
    print("Will create fallback implementations")
    CUSTOM_MODULES_AVAILABLE = False

print("\nImport Summary:")
print(f"- IBM Runtime: {'✓' if IBM_RUNTIME_AVAILABLE else '✗'}")
print(f"- IBM Provider: {'✓' if IBM_PROVIDER_AVAILABLE else '✗'}")
print(f"- Algorithms: {'✓' if ALGORITHMS_AVAILABLE else '✗'}")
print(f"- Custom Modules: {'✓' if CUSTOM_MODULES_AVAILABLE else '✗'}")

✓ qiskit_ibm_runtime imported successfully
✓ qiskit_algorithms imported successfully
ALGORITHMS_AVAILABLE: True
✓ Custom VRP modules imported successfully

Import Summary:
- IBM Runtime: ✓
- IBM Provider: ✗
- Algorithms: ✓
- Custom Modules: ✓


In [114]:
# Cell 2: IBM Quantum Service Configuration
def setup_ibm_quantum_service():
    """Setup IBM Quantum Service with multiple fallback approaches"""
    
    # Method 1: Try qiskit_ibm_runtime
    if IBM_RUNTIME_AVAILABLE:
        try:
            print("Attempting to connect via qiskit_ibm_runtime...")
            
            # Option A: Try without token (if already saved)
            try:
                service = QiskitRuntimeService(channel="ibm_cloud")
                print("✓ Connected using saved credentials")
            except:
                # Option B: Try with token (uncomment and add your token)
                # service = QiskitRuntimeService(
                #     channel="ibm_quantum",
                #     token="YOUR_IBM_QUANTUM_TOKEN_HERE"
                # )
                print("⚠ No saved credentials found. Please set up your IBM Quantum token.")
                print("Run: QiskitRuntimeService.save_account(channel='ibm_quantum', token='YOUR_TOKEN')")
                return None, None
            
            # Get available backends
            backends = service.backends()
            print(f"Found {len(backends)} backends:")
            
            suitable_backends = []
            for backend in backends:
                try:
                    # Try to get basic backend info without status check first
                    qubits = backend.num_qubits
                    backend_name = backend.name
                    
                    # Try to get status, but don't fail if it errors
                    try:
                        status = backend.status()
                        operational = getattr(status, 'operational', True)
                        status_msg = 'Online' if operational else 'Offline'
                    except Exception as status_error:
                        # If status check fails, assume it's operational but note the error
                        operational = True
                        status_msg = f'Status unknown ({str(status_error)[:50]}...)'
                    
                    print(f"- {backend_name}: {qubits} qubits, Status: {status_msg}")
                    
                    # Lower the qubit requirement to work with available backends
                    if qubits >= 5 and operational:  # Changed from 20 to 5
                        suitable_backends.append(backend)
                        
                except Exception as e:
                    print(f"- {getattr(backend, 'name', 'Unknown')}: Error getting info - {str(e)[:100]}...")
            
            if suitable_backends:
                # Sort by number of qubits (descending) to get the best available
                suitable_backends.sort(key=lambda b: b.num_qubits, reverse=True)
                selected_backend = suitable_backends[0]
                print(f"\n✓ Selected backend: {selected_backend.name} ({selected_backend.num_qubits} qubits)")
                return service, selected_backend
            else:
                print("⚠ No suitable backends (5+ qubits) found or accessible")
                # Still return the service for potential local execution
                return service, None
                
        except Exception as e:
            print(f"✗ qiskit_ibm_runtime error: {e}")
    
    # Method 2: Try qiskit_ibm_provider (legacy)
    if IBM_PROVIDER_AVAILABLE:
        try:
            print("\nTrying qiskit_ibm_provider (legacy)...")
            
            try:
                provider = IBMProvider()
                print("✓ Connected using IBMProvider")
            except:
                print("⚠ No saved credentials for IBMProvider")
                print("Run: IBMProvider.save_account(token='YOUR_TOKEN')")
                return None, None
            
            backends = provider.backends()
            print(f"Found {len(backends)} backends via IBMProvider")
            
            suitable_backends = []
            for backend in backends:
                try:
                    config = backend.configuration()
                    qubits = config.n_qubits
                    print(f"- {backend.name()}: {qubits} qubits")
                    
                    if qubits >= 20:
                        suitable_backends.append(backend)
                except Exception as e:
                    print(f"- Backend error: {e}")
            
            if suitable_backends:
                selected_backend = suitable_backends[0]
                print(f"\n✓ Selected backend: {selected_backend.name()} ({selected_backend.configuration().n_qubits} qubits)")
                return provider, selected_backend
            else:
                print("⚠ No suitable backends found")
                return provider, None
                
        except Exception as e:
            print(f"✗ IBMProvider error: {e}")
    
    # Fallback: Local simulation only
    print("\n" + "="*60)
    print("IBM Quantum services not available - using LOCAL SIMULATION")
    print("="*60)
    print("\nTo use IBM Quantum hardware:")
    print("1. Install: pip install qiskit-ibm-runtime")
    print("2. Get token from: https://quantum-computing.ibm.com/")
    print("3. Save token: QiskitRuntimeService.save_account(channel='ibm_quantum', token='YOUR_TOKEN')")
    print("="*60)
    
    return None, None

# Setup IBM Quantum service
service, backend = setup_ibm_quantum_service()

Attempting to connect via qiskit_ibm_runtime...
✓ Connected using saved credentials
Found 2 backends:
- ibm_brisbane: Error getting info - No module named 'qiskit.pulse.parameter_manager'...
- ibm_torino: Error getting info - local variable 'frequency' referenced before assignment...
⚠ No suitable backends (5+ qubits) found or accessible


In [115]:
# Cell 3: VRP Problem Configuration
class VRPConfig:
    """Configuration class for Vehicle Routing Problem"""
    
    def __init__(self, num_customers=5, num_vehicles=2):
        self.num_customers = num_customers
        self.num_vehicles = num_vehicles
        self.max_capacity = 100
        self.depot_location = (0, 0)
        
        # Generate random customer locations and demands
        np.random.seed(42)  # For reproducibility
        self.customer_locations = [(np.random.randint(1, 10), np.random.randint(1, 10)) 
                                 for _ in range(num_customers)]
        self.customer_demands = [np.random.randint(10, 30) for _ in range(num_customers)]
        
        # Calculate distance matrix
        self.distance_matrix = self._calculate_distances()
        
        print(f"VRP Problem Configuration:")
        print(f"- Customers: {num_customers}")
        print(f"- Vehicles: {num_vehicles}")
        print(f"- Customer locations: {self.customer_locations}")
        print(f"- Customer demands: {self.customer_demands}")
    
    def _calculate_distances(self):
        """Calculate Euclidean distance matrix"""
        locations = [self.depot_location] + self.customer_locations
        n = len(locations)
        distances = np.zeros((n, n))
        
        for i in range(n):
            for j in range(n):
                if i != j:
                    distances[i][j] = np.sqrt((locations[i][0] - locations[j][0])**2 + 
                                            (locations[i][1] - locations[j][1])**2)
        return distances

# Initialize problem
vrp_config = VRPConfig(num_customers=6, num_vehicles=2)

VRP Problem Configuration:
- Customers: 6
- Vehicles: 2
- Customer locations: [(7, 4), (8, 5), (7, 3), (7, 8), (5, 4), (8, 8)]
- Customer demands: [12, 11, 21, 15, 11, 10]


In [99]:
# Cell 4: QUBO Formulation
def create_fallback_vrp_qubo(config):
    """Fallback QUBO formulation if custom modules not available"""
    n_customers = config.num_customers
    n_vehicles = config.num_vehicles
    
    # Each customer assigned to exactly one vehicle: n_customers variables
    # Each position in route for each vehicle: n_customers * n_vehicles variables  
    # Total variables needed
    n_vars = n_customers * n_vehicles + n_customers
    
    print(f"Creating fallback QUBO with {n_vars} variables")
    
    # Create a simple QUBO matrix for demonstration
    Q = np.zeros((n_vars, n_vars))
    
    # Add distance costs (simplified)
    for i in range(n_vars):
        for j in range(i+1, n_vars):
            if i != j:
                distance_factor = config.distance_matrix[i % n_customers][j % n_customers]
                Q[i, j] = distance_factor * 0.1
    
    # Add constraint penalties
    penalty = 10
    for i in range(n_customers):
        for j in range(n_vehicles):
            idx = i * n_vehicles + j
            if idx < n_vars:
                Q[idx, idx] += penalty  # Assignment penalty
    
    offset = 0
    return Q, offset

def create_vrp_qubo(config):
    """Create QUBO formulation for VRP"""
    if CUSTOM_MODULES_AVAILABLE:
        try:
            # Use your custom VRP QUBO formulation
            vrp_formulation = VRPQUBOFormulation(
                config.distance_matrix,
                config.num_vehicles,
                config.max_capacity,
                config.customer_demands
            )

            
            # Get QUBO matrix and offset
            Q, offset = vrp_formulation.get_qubo_matrix()
            
            print(f"✓ Using custom QUBO formulation")
            print(f"QUBO Matrix shape: {Q.shape}")
            print(f"Number of qubits required: {Q.shape[0]}")
            print(f"QUBO offset: {offset}")
            
            return Q, offset, vrp_formulation
            
        except Exception as e:
            print(f"⚠ Error with custom QUBO: {e}")
            print("Falling back to simple QUBO...")
    
    # Fallback implementation
    print("Using fallback QUBO formulation...")
    Q, offset = create_fallback_vrp_qubo(config)
    
    print(f"QUBO Matrix shape: {Q.shape}")
    print(f"Number of qubits required: {Q.shape[0]}")
    print(f"QUBO offset: {offset}")
    
    return Q, offset, None

Q_matrix, qubo_offset, vrp_formulation = create_vrp_qubo(vrp_config)
# Track original qubit count
original_qubits = Q_matrix.shape[0]
n_qubits = original_qubits

print(f"\n✓ Total qubits needed: {n_qubits}")

# Ensure we have at least 20 qubits for demonstration
if n_qubits < 20:
    print(f"⚠ Current problem uses {n_qubits} qubits. Expanding to 20+ qubits...")
    # Expand the problem to ensure 20+ qubits
    target_qubits = 22
    Q_expanded = np.zeros((target_qubits, target_qubits))
    Q_expanded[:n_qubits, :n_qubits] = Q_matrix
    
    # Add some random coupling to additional qubits
    for i in range(n_qubits, target_qubits):
        for j in range(i):
            Q_expanded[j, i] = np.random.random() * 0.1

    print(f"✓ Added random couplings to expanded qubits:")
    for i in range(target_qubits - 2, target_qubits):
        print(f"  - Qubit {i}: Coupled with previous qubits")
    
    Q_matrix = Q_expanded
    n_qubits = target_qubits
    print(f"✓ Expanded to {n_qubits} qubits")

print(f"Final qubit count: {n_qubits}")

⚠ Error with custom QUBO: VRPQUBOFormulation.__init__() takes from 3 to 4 positional arguments but 5 were given
Falling back to simple QUBO...
Using fallback QUBO formulation...
Creating fallback QUBO with 18 variables
QUBO Matrix shape: (18, 18)
Number of qubits required: 18
QUBO offset: 0

✓ Total qubits needed: 18
⚠ Current problem uses 18 qubits. Expanding to 20+ qubits...
✓ Added random couplings to expanded qubits:
  - Qubit 20: Coupled with previous qubits
  - Qubit 21: Coupled with previous qubits
✓ Expanded to 22 qubits
Final qubit count: 22


In [100]:
# Cell 5: Convert QUBO to Ising Hamiltonian
def qubo_to_ising(Q, offset=0):
    """Convert QUBO matrix to Ising Hamiltonian"""
    n = Q.shape[0]
    
    # Create Pauli operators
    pauli_list = []
    
    # Linear terms (diagonal)
    for i in range(n):
        if Q[i, i] != 0:
            pauli_str = ['I'] * n
            pauli_str[i] = 'Z'
            pauli_list.append((''.join(pauli_str), Q[i, i] / 2))
    
    # Quadratic terms (off-diagonal)
    for i in range(n):
        for j in range(i + 1, n):
            if Q[i, j] != 0:
                pauli_str = ['I'] * n
                pauli_str[i] = 'Z'
                pauli_str[j] = 'Z'
                pauli_list.append((''.join(pauli_str), Q[i, j] / 4))
    
    # Constant term adjustment
    constant = offset + np.sum(np.diag(Q)) / 2
    
    # Create SparsePauliOp
    if pauli_list:
        hamiltonian = SparsePauliOp.from_list(pauli_list)
    else:
        # Create identity if no terms
        hamiltonian = SparsePauliOp.from_list([('I' * n, 1.0)])
    
    return hamiltonian, constant

# Convert to Ising Hamiltonian
hamiltonian, ising_offset = qubo_to_ising(Q_matrix, qubo_offset)
print(f"Ising Hamiltonian created with {len(hamiltonian)} terms")
print(f"Ising offset: {ising_offset}")

Ising Hamiltonian created with 225 terms
Ising offset: 60.0


In [102]:
# Cell 6: QAOA Circuit Construction
def create_qaoa_circuit(hamiltonian, p_layers=2):
    """Create QAOA circuit for the given Hamiltonian"""
    
    # Create QAOA ansatz
    qaoa_ansatz = QAOAAnsatz(hamiltonian, reps=p_layers)
    
    print(f"QAOA Circuit Statistics:")
    print(f"- Qubits: {qaoa_ansatz.num_qubits}")
    print(f"- Parameters: {qaoa_ansatz.num_parameters}")
    print(f"- Layers (p): {p_layers}")
    print(f"- Gates: {qaoa_ansatz.count_ops()}")
    
    return qaoa_ansatz

# Create QAOA circuit
p_layers = 2
qaoa_circuit = create_qaoa_circuit(hamiltonian, p_layers)

QAOA Circuit Statistics:
- Qubits: 22
- Parameters: 4
- Layers (p): 2
- Gates: OrderedDict([('QAOA', 1)])


In [104]:
# Cell 7: Optimizer Configuration
import numpy as np
from qiskit_algorithms.optimizers import SPSA, COBYLA

class SimpleOptimizer:
    """Simple optimizer implementation if qiskit_algorithms not available"""

    def _init_(self, maxiter=50, method='SPSA'):
        self.maxiter = maxiter
        self.method = method
        self.callback = None
        self.history = []

    def minimize(self, cost_function, initial_point):
        """Simple optimization using scipy or fallback random search"""
        try:
            from scipy.optimize import minimize

            def objective(params):
                try:
                    return cost_function(params)
                except Exception:
                    return float('inf')

            if self.method == 'SPSA':
                best_params = initial_point.copy()
                best_cost = objective(best_params)

                for i in range(self.maxiter):
                    scale = 0.1 / (1 + 0.05 * i)
                    perturbation = np.random.normal(0, scale, len(best_params))
                    trial_params = best_params + perturbation
                    trial_cost = objective(trial_params)

                    if trial_cost < best_cost:
                        best_params = trial_params
                        best_cost = trial_cost

                    if self.callback and i % 10 == 0:
                        self.callback(best_params)
                    self.history.append((i, best_cost))

                return type('Result', (), {
                    'x': best_params,
                    'fun': best_cost,
                    'nfev': self.maxiter
                })()

            else:
                result = minimize(objective, initial_point, method='Nelder-Mead',
                                  options={'maxiter': self.maxiter})
                return result

        except ImportError:
            print("⚠ scipy not available, using random search")
            best_params = initial_point.copy()
            best_cost = cost_function(best_params)

            for i in range(self.maxiter):
                trial_params = initial_point + np.random.normal(0, 0.1, len(initial_point))
                trial_cost = cost_function(trial_params)

                if trial_cost < best_cost:
                    best_params = trial_params
                    best_cost = trial_cost

            return type('Result', (), {
                'x': best_params,
                'fun': best_cost,
                'nfev': self.maxiter
            })()

def setup_optimizers():
    """Setup different optimizers with fallback implementations"""
    optimizers = {}
    ALGORITHMS_AVAILABLE = True

    try:
        # SPSA callback
        optimizers['SPSA'] = SPSA(
            maxiter=100,
            callback=lambda x, y, *args: print(
                f"SPSA Iteration: {x}, Cost: {float(np.atleast_1d(y)[0]):.6f}")
        )

        # COBYLA callback with counter
        iteration_counter = {'count': 0}

        def cobyla_callback(x, y):
            iteration_counter['count'] += 1
            cost = float(np.atleast_1d(y)[0])
            print(f"COBYLA Iteration: {iteration_counter['count']}, Cost: {cost:.6f}")

        optimizers['COBYLA'] = COBYLA(
            maxiter=100,
            callback=cobyla_callback
        )

        print("✓ Using qiskit_algorithms optimizers")

    except Exception as e:
        print(f"⚠ Error setting up qiskit optimizers: {e}")
        ALGORITHMS_AVAILABLE = False

    if not ALGORITHMS_AVAILABLE:
        optimizers['SPSA'] = SimpleOptimizer(maxiter=100, method='SPSA')
        optimizers['COBYLA'] = SimpleOptimizer(maxiter=100, method='COBYLA')
        optimizers['SCIPY'] = SimpleOptimizer(maxiter=50, method='Nelder-Mead')
        print("✓ Using fallback optimizers")

    return optimizers

# Initialize optimizers
optimizers = setup_optimizers()
print(f"Available optimizers: {list(optimizers.keys())}")

✓ Using qiskit_algorithms optimizers
Available optimizers: ['SPSA', 'COBYLA']


In [107]:
def create_cost_function(qaoa_circuit, hamiltonian, estimator):
    """Create cost function for QAOA optimization"""

    def cost_function(params):
        """Evaluate cost function for given parameters"""
        try:
            # Bind parameters to circuit
            bound_circuit = qaoa_circuit.assign_parameters(params)

            # Run circuit using IBM Estimator
            job = estimator.run(bound_circuit, observables=hamiltonian)
            result = job.result()

            # Extract expectation value
            expectation_value = result.values[0]

            return expectation_value

        except Exception as e:
            print(f"Error in cost function: {e}")
            return float('inf')

    return cost_function

In [108]:
# Cell 9: Local Simulation Setup
from qiskit.primitives import Sampler as LocalSampler

def setup_local_simulation():
    """Setup local simulation environment"""
    
    # Create local simulator
    simulator = AerSimulator()
    local_sampler = LocalSampler()
    
    print("Local simulation environment setup complete")
    print(f"Simulator backend: {simulator.name}")
    
    return simulator, local_sampler

local_simulator, local_sampler = setup_local_simulation()

Local simulation environment setup complete
Simulator backend: aer_simulator


In [110]:
#cell 10
from qiskit_ibm_runtime import Sampler

def setup_ibm_execution(service, backend, local_sampler):
    """Setup IBM Quantum execution environment"""

    if service is None or backend is None:
        print("✓ IBM Quantum service not available, using local simulation only")
        return None, local_sampler

    try:
        if IBM_RUNTIME_AVAILABLE and hasattr(service, 'backends'):
            # qiskit_ibm_runtime approach
            ibm_sampler = Sampler(backend=backend.name)
            print(f"✓ IBM Runtime execution setup complete")

        elif IBM_PROVIDER_AVAILABLE:
            # qiskit_ibm_provider approach - fallback to local
            print(f"✓ IBM Provider setup (will use local execution)")
            ibm_sampler = local_sampler

        else:
            print("⚠ No IBM Quantum execution available")
            return None, local_sampler

        try:
            backend_name = backend.name if hasattr(backend, 'name') else backend.name()
            backend_qubits = backend.num_qubits if hasattr(backend, 'num_qubits') else backend.configuration().n_qubits
            print(f"Backend: {backend_name}")
            print(f"Qubits: {backend_qubits}")
        except:
            print("Backend info not accessible")

        return backend, ibm_sampler

    except Exception as e:
        print(f"⚠ Error setting up IBM execution: {e}")
        print("Falling back to local simulation")
        return None, local_sampler

# Setup IBM execution environment
ibm_backend, ibm_sampler = setup_ibm_execution(service, backend, local_sampler)

✓ IBM Quantum service not available, using local simulation only
